# pyRTC Example: Simulated Single Conjugate AO system with Pyramid Wavefront Sensor

This example will take you through how to set-up a basic SCAO system in pyRTC which uses the OOPAO simulation software to simulate the necessary AO hardware components.

### IMPORTANT NOTE BEFORE STARTING

**This notebook runs all of the AO components inside a single process. Because of the python Global Interpretter Lock, this will lead to SLOW performance, but will easier to debug. If you would like to run each computation as an independent process you will need to use the Hardware Launcher tool. Please see pywfs_example_OOPAO_pipeline.py for instructions** 

In [ ]:
#Import the basics
import numpy as np
import yaml
import matplotlib.pyplot as plt

#Import hardware classes
from pyRTC.hardware.OOPAOInterface import OOPAOInterface
from pyRTC.SlopesProcess import SlopesProcess
from pyRTC.Pipeline import *
from pyRTC.Loop import Loop

import time

In [ ]:
"""
Shared memory in python is a bit annoying, we are required to unlink it from the garbage collector
so that it will stick around in between runs, however sometime you can get into a situation where 
the SHM is not intialized properly. Usually you will see an error like: 
TypeError: buffer is too small for requested array

To reset a SHM you can run the following code. Note: it will throw some garbage collector errors.
"""
# shm_names = ["wfs", "wfsRaw", "wfc", "wfc2D", "signal", "signal2D", "psfShort", "psfLong"] #list of SHMs to reset
# clear_shms(shm_names)

pyRTC uses YAML config files to specify all of the variables for each component of your AO system. Setting up the config file properly is very important in order to have pyRTC run correctly.

For this example, we will be using the config file: pywfs_OOPAO_config.yaml

In [ ]:
# Load the configuration file
def read_yaml_file(file_path):
    with open(file_path, 'r') as file:
        conf = yaml.safe_load(file)
    return conf

#Now we can read our YAML config file 
conf = read_yaml_file("pywfs_OOPAO_config.yaml")

#And separate it into sections for each of our AO loop components
confLoop = conf["loop"]
confWFS = conf["wfs"]
confWFC = conf["wfc"]
confPSF = conf["psf"]
confSlopes = conf["slopes"]

print(confLoop)
print(confWFS)
print(confWFC)
print(confPSF)
print(confSlopes)

We have defined an OOPAO interface object which sets up an SCAO loop. This object is a wrapper for the following sub classes:

- _OOPAOScienceCamera
- _OOPAOSlopesProcess
- _OOPAOWFCorrector
- _OOPAOWFSensor

Each of these subclasses is a child of a pyRTC core class, which adds the interfacing and logic required by the OOPAO package. There are several core pyRTC classes, which each represent abstract versions of standard AO components. Some examples of pyRTC core classes include:

- WavefrontSensor
- WavefrontCorrector
- SlopesProcess
- ScienceCamera

The OOPAO interface is a wrapper for these objects which initializes them given the user provided configuration. Right now, this is limited to SCAO systems, but there is no reason that it could not be easily modified to support multiple wavefront sensors or correctors.

In [ ]:
"""
Create the OOPAO simulation interface object 
Running this cell will initialize the dm, wfs, psf, and slopes objects, 
but will not start their real time computations. This inialization includes
the creation of the Shared Memory Objects, and the simulation inialization.
"""
sim = OOPAOInterface(conf=conf, param=None)
wfs, dm, psf = sim.get_hardware()

"""
It's important to set the full basis and number of possible modes before
initializing the loop object. Here I define a KL basis for the system
"""
from OOPAO.calibration.compute_KL_modal_basis import compute_KL_basis

NUM_MODES = 100 #confWFC["numModes"] #must be less than total KL modes

M2C_KL = compute_KL_basis(sim.tel, sim.atm, sim.dm)
dm.setM2C(M2C_KL[:,:NUM_MODES])

"""
"""
slopes = SlopesProcess(conf=conf)

""" 
"""
#Initialize our AO loop object
loop = Loop(conf=conf)


In [ ]:
"""
Start the processes. Here the real-time computations selected in
the config will begin.
"""
dm.start()
dm.flatten()

wfs.start()
slopes.start()

print(sim.dm.OPD.shape)
psf.start()

### IMPORTANT: REAL-TIME VIEWING 

At this point, you can look at the shared memory objects to see what is happening in real time as we run the AO loop.

First, navigate on a terminal to the pyRTCView Folder: 

```cd INSTALLTION_LOCATION/pyRTC/pyRTCView```

Now run the viewer for whatever shared memory you would like to look at. For the default simulation run:

```
python pyRTCView.py wfs &
python pyRTCView.py wfc2D &
python pyRTCView.py signal2D &
python pyRTCView.py psfShort &
```

In [ ]:
#Remove the atmosphere from the simulation
sim.removeAtmosphere()

loop.pokeAmp = 1e-7

#Compute the IM, blocking
loop.computeIM()

#Add the atmosphere back to the simulation
sim.addAtmosphere()

In [ ]:
plt.imshow(sim.dm.OPD)

In [ ]:
loop.plotIM()

In [ ]:
dm.flatten()
time.sleep(1e-2)
loop.setGain(0.3)
loop.start()
time.sleep(10)
loop.stop()
dm.flatten()

In [ ]:
plt.imshow(sim.wfs.signal_2D)
plt.colorbar()
plt.show()

plt.imshow(slopes.signal2D.read_noblock_safe())
plt.colorbar()
plt.show()